# Diamant NV center 
**Het Zeeman-effect waanemen**
practicum handleiding

Versie: mei 2025
Door: Rutger Ockhorst, Lodewijk Koopman en Mark Melotto, aanpassingen Silvester Infante Ferreira

Contact docenten of leerlingen: qlab@tudelft.nl
<br><br>
<img src="./media/nv/QDNL.png" width="200" align="left"/><img src="./media/nv/TU_Delft_Logo.svg" width="150" align="right"/>
<br>
<br>
<br>
Financiering: Quantum Delta NL / Nationaal Groeifonds



# 1. Inleiding
In dit experiment bestuderen we de invloed van een extern magnetisch veld op de spintoestand van elektronen. Concreet meten we de fluorescentie van stikstof-gat defecten in diamant(poeder) en de hoeveelheid fluorescentie verandert als we een magneet in de buurt van het poeder houden.

Deze versie van de proef is vooral een kwalitatieve verkenning. Een uitgebreidere versie is in de maak, waarbij met hoge precisie de waarde van het magnetisch veld bepaald kan worden. De defecten in het diamantpoeder zijn een voorbeeld van een quantumsensor. Er is een NLT-module voor havo in de maak, werktitel __Quantum Ready__, waarbij in ieder hoofdstuk een stukje theorie achter deze proef wordt behandeld (licht, energie, fluorescentie, magnetisme).

Hieronder volgt eerst een beknopte theoretische toelichting op en daarna instructies voor het uitvoeren van de proef. 

# 2. Theorie

### 2.1 NV-centers in een diamantrooster
<div>
    <img src="./media/nv/rooster.jpg" width="250" align="right"/>
</div>
<br>
<br>

Diamant is opgebouwd uit koolstofatomen in een rooster met een zgn. face centered cubic structuur. Behalve koolstofatomen kunnen er in het rooster ook defecten voorkomen, zoals gaten of andere atoomsoorten. Bij deze proef wordt gebruik gemaakt van stikstof-gat (nitrogen-vacancy of NV) defecten. Deze defecten kunnen in natuurlijke diamant voorkomen maar ook in het lab gemaakt worden. Kort door de bocht: de diamant groeit in een atmosfeer van stikstof gas, waardoor er stikstofatomen in het rooster verstrikt raken. Later worden er met gaten in het rooster geschoten, bijvoorbeeld met een elektronen bundel. Door de diamant te verhitten tot zo'n 900° Celsius migreren de gaten door het rooster heen naar de stikstofatomen. Zo ontstaat een stikstof-gat-paar (NV center). Diamanten met een hoge concentratie NV-centers zijn roze.

Er wijzen vijf ongebonden elektronen naar het gat: drie van de naburige koolstofatomen en twee van het stikstofatoom (stikstof staat één plaats rechts van koolstof in het periodiek systeem). Het blijkt energetisch gunstig om een zesde elektron in te vangen in het gat. Zo ontstaat een negatief geladen NV$^{-}$-center. In deze toestand kan het NV-center worden gebruikt als qubit of magneetsensor.


#### 2.1. 3D rooster bekijken
Wil je het diamantrooster met NV center beter bekijken, voer dan met onderstaande cel uit. Dit kan door rechtsboven op de raket te drukken (live code execution) en vervolgens op run te klikken.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Line3DCollection
from itertools import product, combinations
from ipywidgets import interact, IntSlider


coordinates = np.array([
    [0.5, 0, 0.0], [0, 0.5, 0.0], [1, 0.5, 0.0], [0.5, 1, 0.0],
    [0.25, 0.75, 0.25],
    [0, 0, 0.5], [1, 0, 0.5], [0, 1, 0.5], [1, 1, 0.5],
    [0.25, 0.25, 0.75], [0.75, 0.75, 0.75], 
    [0.5, 0, 1], [0, 0.5, 1], [1, 0.5, 1], [0.5, 1, 1],
])

N = np.array([0.75, 0.25, 0.25])
V = np.array([0.5, 0.5, 0.5])
all_coords = np.concatenate((coordinates, [V], [N]), axis=0)

def distance(p1, p2):
    return np.linalg.norm(p1 - p2)

def find_nearest_neighbors(coords, threshold=0.6):
    pairs = []
    for i in range(len(coords)):
        for j in range(i+1, len(coords)):
            if distance(coords[i], coords[j]) <= threshold:
                pairs.append((i, j))
    return pairs

def plot_atoms(elev=30, azim=30):
    fig = plt.figure(figsize=(6,6))
    ax = fig.add_subplot(111, projection='3d')

    # Plot carbon atoms
    for coord in coordinates:
        ax.scatter(*coord, color='dimgrey', edgecolor='black', s=100)

    # Special points
    ax.scatter(*V, color='white', edgecolor='black', s=150)
    ax.scatter(*N, color='blue', edgecolor='black', s=150)

    # Draw bonds
    neighbors = find_nearest_neighbors(all_coords)
    for i, j in neighbors:
        a1, a2 = all_coords[i], all_coords[j]
        is_nv = np.any(np.all(a1 == V, axis=0)) or np.any(np.all(a2 == V, axis=0)) \
                or np.any(np.all(a1 == N, axis=0)) or np.any(np.all(a2 == N, axis=0))
        linestyle = 'dashed' if is_nv else 'solid'
        ax.plot([a1[0], a2[0]], [a1[1], a2[1]], [a1[2], a2[2]], color='green', linestyle=linestyle)

    # Draw cube
    for s, e in combinations(np.array(list(product([0, 1], repeat=3))), 2):
        if np.sum(np.abs(s-e)) == 1:
            ax.plot3D(*zip(s, e), color="blue", linestyle='dashed')

    ax.view_init(elev=elev, azim=azim)
    ax.set_proj_type('ortho')
    ax.set_box_aspect([1,1,1])
    ax.axis('off')
    plt.show()


import matplotlib as mpl
# Monkeypatch als _get ontbreekt (werkt als tijdelijke oplossing)
if not hasattr(mpl.rcParams, "_get"):
    mpl.rcParams._get = mpl.rcParams.get
# ...existing code...

# Interactieve sliders met betere beginstand
interact(plot_atoms, 
         elev=IntSlider(value=12, min=0, max=90, step=1, description='Elevatie'), 
         azim=IntSlider(value=82, min=0, max=360, step=1, description='Azimut')
         );



### 2.2 Spintoestanden en energieniveaus
Elektronen hebben een spin van ±½, vaak spin down (↓) en spin up (↑) genoemd. De elektronen in het NV-center zijn met elkaar verstrengeld en gedragen zich gezamenlijk als een systeem met spin 1. Dit levert (netto) de volgende toestanden op:

Een singlet toestand: $\frac{1}{\sqrt{2}}$ (↓↑ − ↑↓)

Drie triplet toestanden:
*  $\frac{1}{\sqrt{2}}$ (↓↑ + ↑↓)
*  ↑↑
*  ↓↓

> 1) __Vraag:__ wat zijn de spin-waardes van ieder van deze vier toestanden?
 
In het onderstaande energieniveauschema zijn de grondtoestand en de aangeslagen niveaus van de triplet toestanden in de paarse vlakken aangegeven. De singlet toestand is in een geel vlak afgebeeld. Zoals je kunt zien is de energie van de singlet toestand hoger in energie dan de triplet-grondtoestanden. (N.B.: de overgangen tussen de toestanden kun je voor nu negeren, deze worden in 2.3 toegelicht).

<div>
<img src="./media/nv/NV-energy-levels.svg" width="350"/>
</div>
<center>(Diagram door BonPhire, BY-SA 4.0, https://en.wikipedia.org/wiki/File:NV-energy-levels.svg)</center>

Zonder extern magnetisch veld hebben de energieniveaus van de ↑↑-toestand (m$_{s}$ = +1) en de ↓↓-toestand (m$_{s}$ = −1) dezelfde energie. Is er wel een extern magneetveld, dan splitsen deze energieniveaus op: dit wordt zeemansplitsing genoemd (naar Pieter Zeeman). De mate van opsplitsing hangt af van de sterkte van het magneetveld in de richting van het NV-centrum.

> 2)  __Vraag__: waarom verandert de laagste energietoestand niet onder invloed van een magnetisch veld?

Hoe sterker het magnetisch veld, hoe dichter de energie van de m$_{s}$=-1 toestand bij de grondtoestand komt te liggen. Bij een magnetisch veld van 102.5 mT snijden de twee energieniveaus elkaar en bij een nog groter veld wordt dus de m$_{s}$=-1 toestand de grondtoestand. Dit is weergegeven in onderstaand diagram.

<img src="./media/nv/mixing.png" width="550"/>
<center>(Diagram door Patel et al., https://arxiv.org/abs/2306.17531)</center>

Doordat de scheiding tussen de energietoestanden niet heel groot is (zeker in vergelijking met k$_{B}$T bij kamertemperatuur) zul je in (bij benadering) een superpositie mogen verwachten tussen de m$_{s}$=0 en m$_{s}$=-1 toestanden. Met het aanleggen van een extern magneetveld wordt de bijdrage van de m$_{s}$=-1 toestand in die superpositie steeds groter.





### 2.3 Fluorescentie
Een NV center fluoresceert onder invloed van groen (laser)licht. Dat is op onderstaande foto goed te zien. Je ziet een buisje met diamantpoeder met een hoge concentratie NV centers (3.5 ppm). Rechts wordt het buisje beschenen met een groene laser. De fluorescentie is met het blote oog te zien.

<img src="./media/nv/Fluorescentie.jpg" width="450"/>

Op onderstaande foto zie je je de fluorescerende diamantjes nog verder ingezoomd.

<img src="./media/nv/diamant.jpg" width="450"/>

Vanuit zowel de m$_{s}$ = 0 als vanuit de m$_{}s$ = ±1 toestand wordt het systeem door het groene licht aangeslagen naar een band van energietoestanden. Omdat de spin behouden is verliest het systeem eerst energie via niet-radiatieve, interne overgangen en komt het terecht in de aangeslagen triplettoestand met overeenkomend m$_{s}$-getal. Van daaruit valt het terug naar de grondtoestand onder uitzending van een rood foton met een golflengte van tussen de 650 en 750 nm (fluorescentie). Deze overgangen zijn weergegeven met de dikke groene en rode pijlen in figuur 1.

<img src="./media/nv/NV-transitions.svg" width="400"/>
(Diagram door BonPhire, BY-SA 4.0, https://commons.wikimedia.org/wiki/File:NV-transitions.svg)

Er is daarnaast een kleine kans dat het NV center terugvalt via de tussengelegen singlet grondtoestand van de singlet toestand ($^{1}A_{1}$ in de figuur). In dit geval is spin niet behouden en dat maakt deze overgangen minder waarschijnlijk. Bij dit proces komt geen (zichtbaar) licht vrij. Dit niet-radiatieve verval is waarschijnlijker vanuit de m$_{s}$ = ±1 toestand dan vanuit de m$_{s}$ = 0 toestand. Zoals in 2.2 is beschreven, neemt bij een groter extern magneet veld de kans toe, dat het NV center zich in de m$_{s}$ = -1 toestand bevindt. En daarmee neemt de hoeveelheid fluorescentie dus af. Zo geeft de mate van fluorescentie informatie over de spintoestand waarin het NV centrum zich bevindt en daarmee kan in principe de grootte van het extern magnetisch veld worden bepaald.

# 3. Proefopstelling



### 3.1 Kastje
<div></div>
De proefopstelling bestaat uit een 3D geprint, lichtdicht doosje. In het doosje zie je aan het ene uiteinde een rood plaatje acrylaat. Achter het acrylaat bevindt zich een TSL2591 sensor voor Arduino, die met een fotodiode lichtintensiteit meet. Deze heeft een heel groot dynamisch bereik. Dat wil zeggen dat hij zeer gevoelig is voor (veranderingen) in lichtintensiteit. Het nadeel hiervan is dat de sensor (relatief) langzaam (hooguit 8 metingen per seconde).
Hieronder een twee foto's.

<div>
    <img src="./media/nv/newbox-closed.jpeg" width="200"/>
</div>
<div>
    <img src="./media/nv/newbox-open2.jpeg" width="200"/>
</div>

Aan de voorkant van het acrylaat is diamantpoeder met een hoge concentratie NV-centers gelijmd m.b.v. secondelijm. Vanaf de opening aan de andere kant van het doosje schijn je __STRAKS__ met een groene laser op het diamantpoeder.

<div>
    <img src="./media/nv/newbox-diamant.jpeg" width="200"/>
</div>

> 3) __Vraag:__ waarom is het acrylaat rood?

Het kastje zal niet 100% licht dicht zijn. Om toch zo weinig mogelijk achtergrondsignaal te hebben, probeer de deksel goed te sluiten.
<br><br>
Er is ook een andere versie van dit doosje, met plaats voor twee Helmholtz-spoelen. Het voordeel hiervan is dat hier geen permanente magneet bij nodig is en de sterkte van het magnetischveld eenvoudig ingesteld kan worden met een voeding (stroombron). Hieronder twee afbeeldingen. 
<div>
    <img src="./media/nv/coil-box-closed.jpeg" width="200"/>
</div>
<div>
    <img src="./media/nv/coil-box-open.jpeg" width="200"/>
</div>

### 3.2 Laser
We gebruiken een 520 nm laser van Thorlabs met een USB-aansluiting. Wacht nog even met aansluiten. Klem de laser in de bevestiging in het kastje. Met de laser in het doosje sluit je de USB-kabel aan op een vrije poort in de computer (of een andere USB-spanningsbron). De laser gaat pas aan enkele seconden nadat je op het zwarte knopje op de USB-stekker hebt geklikt. De laser is ingeklemd gericht op het diamantpoeder. Door de hoge brekingsindex van diamant zie je de korrels duidelijk oplichten / schitteren. 
__Zet de laser weer UIT__: we gaan zo eerst experimenteren met een donker doosje. 

### 3.3 Dekseltje
Doe nu het dekseltje op het doosje om de boel af te sluiten. Zorg dat de kabeltje netje naar buiten komen en niet kapot gaan. Deze kabeltjes gaan we nu aansluiten (3.4). Als je de deksel later weer van de doos haalt, koppel dan de Arduino (zie 3.4) even af. Anders moet je mogelijk lang wachten voordat de sensor weer stabiel is.

### 3.4 Arduino
De Arduino is al voorgeprogrammeerd en hoeft in principe alleen te worden aangesloten op de computer met een USB-kabel. Desalniettemin kan de software hier worden gedownload: {Download}`software<./media/nv/Arduino-software.zip>`

De draden van de TSL2591 steken aan de zijkant uit het doosje. Deze moeten als volgt worden aangesloten op de Arduino: *(zorg dat bij het aansluiten de USB kabel niet is aangesloten)*

*  Geel in SCL of A5 bij een UNO
*  Blauw in SDA of A4 bij een UNO
*  Zwart in GND
*  Rood in 5V

De opstelling is beschikbaar in het **QuantumLab** bij de TU Delft voor profielwerkstukken havo (!) en vwo. Er is ook een NLT-module beschikbaar over quantum sensoren waar deze proef ook onderdeel van is. Voor meer informatie kun je contact opnemen met Rutger Ockhorst via qlab@tudelft.nl.

# 4. Experimenteren
Metingen kun je opstarten vanuit **hoofdstuk 5**. Voel je vrij om te experimenteren, maar we doen hieronder een paar suggesties om je op weg te helpen.

Voordat je begint met meten: controleer steeds of je een stabiel signaal hebt. Soms kan de sensor enige tijd nodig hebben om te stabiliseren, bijvoorbeeld als het doosje net is open geweest en de sensor (over)belicht is. Stabiel is een signaal dat enigszins een horizontale lijn volgt en niet continu oploopt. Een beetje is niet erg. Dit kan circa 1 minuut duren. 

### 4.1 Gevoeligheid voor achtergrondlicht bepalen
Voer een meting uit van 60 seconden met een dicht doosje, laser uit. Dit zou een signaal van 0 tot ongeveer 100 op mogen leveren. Lager is beter. Kijk of het kastje erg gevoelig is voor variaties in de omgeving. Hou je handen bijvoorbeeld eens om het kastje heen of schuif je stoel eens naar achter. In het ideale geval heeft dit geen invloed.

### 4.2 Fluorescentiesignaal zonder extern magnetisch veld bepalen
Zet de laser aan en doe een meting van 30 seconden. bij een goed uitgelijnde laser kun je een fluorescentiesignaal verwachten van 10k tot 30k. Controleer of het signaal stabiel is (fluctuaties van minder dan een procent zijn ok). Het kan soms voorkomen dat de sensor langer nodig heeft om te stabiliseren. Dan zie je bijvoorbeeld een continu oplopend signaal. Herhaal de meting dan een aantal keer, misschien na stabiliseren van de laser of verplaatsen van het kastje, tot je een stabiel signaal ziet. Eventueel kun je de Arduino resetten. Haal de USB kabel een paar seconden los, voor hem weer aan te sluiten. Maak vervolgens opnieuw verbinding in de software (zie bij hoofstuk Meten).

### 4.3 Invloed van het magnetisch veld waarnemen
<img src="./media/nv/magnet.png"/>
<br>
Als je een stabiel signaal hebt kun je een meting doen met een extern magnetisch veld. 
Er zijn twee versies van dit experiment. 

#### 4.3.1 Losse magneet
Voor de versie met losse **neodynium magneet**:
Let op: de magneten die we hier gebruiken zijn sterk tot __zeer__ sterk. Houd je pinpas, laptop etc. dus uit de buurt. Controleer je werkplek ook op losliggende metalen onderdelen die door de magneet aangetrokken kunnen worden (misschien met je hand er tussen: auw!). Ben je klaar met de magneet, berg 'm dan op in de doos, laat ze absoluut niet los op tafel liggen.

Er staan streepjes aan de zijkant van het doosje waar je de magneet op verschillende afstanden van het sample kunt houden. Onderzoekjes die je zou kunnen doen:

*  hoe verandert het fluorescentiesignaal met de afstand tussen magneet en diamantpoeder?
*  welk van de beschikbare magneten is sterker?
*  ...
#### 4.3.2 Elektromagneten
Deze versie is voorzien van twee elektromagneten (spoelen), die fungeren als Helmholtz spoelen en dus zorgen voor een sterk homogeen magnetisch veld ter hoogte van het sample.
De twee spoelen zijn zo geplaatst en draadgewonden dat de stroom in beide spoelen dezelfde kant op loopt. Dit zorgt voor een magnetisch veld in dezelfde richting, met daar midden tussenin, een zeer homogeen magnetische veld.
De spoelen worden middels de banana stekkers aangesloten op een externe voeding. Zorg dat de voeding uitstaat tijdens het aansluiten. Sluit de twee rode kabels aan op de pluspool en de twee zwarte op de minpool van voeding. Zet de voeding op de maximale spanning en stroomsterkte voor deze aan te zetten. Zie verder: het hoofdstuk Meten. <br><br>
**LET OP: De voeding niet langer dan een minuut achter elkaar aan laten staan. De spoelen worden erg warm en kunnen smelten.**  



# 5. Meten

### 5.1 Meting met opstelling
Met de link hieronder wordt via javascript code de lichtsterkte die de Arduino meet weergegeven. Op de y-as staat de ruwe data van de sensor, dus in zeker opzicht _arbitrary units_, maar in principe is dit om te rekenen naar de lichtsterkte in lux. De sensor kan signalen tussen de 188 μlux en 88 klux meten. 

* Sluit de USB poort aan. 
* Open onderstaande link met een **Chrome** of Edge browser en 
* Geef de USB-poort toestemming.  
* Bedenk welke meting je wilt doen (zie hoofdstuk 4). 
* Vergeet niet dat de laser aan moet staan (knopje op USB connector), voor meting 4.2 en 4.3.
* Laat de meting circa 20 seconden lopen voordat je gaat onderzoeken met een (elektro)magneet. Zie hoofdstuk 4 hierboven voor meet-mogelijkheden.


<a href="./media/nv/NV_live_applet_nieuw.html" target="_blank">Open NV Live Applet</a>
<br><br>
Als alternatief is het ook mogelijk de html-pagina hieronder te downloaden en vervolgens te openen in een **Chrome** of Edge browser.
{Download}`software<./media/nv/NV_live_applet_nieuw.html>`



### 5.2 Voorbeeld van een meting
Hieronder een voorbeeld van een meetreeks. Verticaal staat (in willekeurige eenheid) de intensiteit van de fluorescentie. Rond 10 seconden wordt er een magneet bij het poeder gebracht en gehouden. Rond 22 seconden wordt de magneet weer (rustig) weg gehaald.
<img src="./media/nv/VoorbeeldMeting.png" width="400"/>

# 6. Mogelijke onderzoeksvragen voor vervolg onderzoek
Hier enkele voorbeelden van onderzoeksvragen:
* Meet de achtergrondruis onder donkere omstandigheden (zonder laser of fluorescentie).
* Meet de transmissie van groen laserlicht door 3 mm rood acrylaat (zonder diamant).
* (mogelijke bron van systematische fout)
* Meet de fluorescentie van diamant (controleer of het fluorescentiesignaal te onderscheiden is van achtergrond en transmissie).
* Vergelijk met behulp van fluorescentie de sterkte van verschillende neodymiummagneten op dezelfde afstand van het diamantmonster.
* Meet de fluorescentie voor dezelfde neodymiummagneet op verschillende afstanden van het diamantmonster.
* Bepaal de gevoeligheid van de diamantsensor voor variaties in de stroom door een elektromagneet.
* Gebruik bekende magnetische veldsterktes om je opstelling te kalibreren zodat het een kwantitatieve sensor wordt.

# 7. Klaar?
Ruim dit experiment op en ga verder met <a href="./nv2.html">het tweede NV experiment</a>.